# Modelado avanzado y calibración de modelos

En esta sección realizamos el **modelado avanzado** sobre el dataset limpio con *features* ya generadas (`Cleaned_Featured_Dataset.csv`).  
Los objetivos principales son:

- Probar **tres modelos de clasificación**:
  - Regresión logística (tuneada con GridSearchCV)
  - Random Forest (tuneado)
  - Gradient Boosting (tuneado)
- Utilizar **validación cruzada (CV=5)** para seleccionar los mejores hiperparámetros.
- Comparar el desempeño de los modelos usando:
  - Accuracy, Precision, Recall, F1, AUC
  - Reporte de clasificación
- Aplicar **calibración de probabilidades** (Platt / Isotónica) al mejor modelo (Random Forest) y comparar el **Brier score** y el AUC antes y después de calibrar.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, classification_report,
    brier_score_loss
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV

RANDOM_STATE = 42

print("Cargando dataset limpio con features...")
data = pd.read_csv("Cleaned_Featured_Dataset.csv")
print("Filas:", len(data))



Cargando dataset limpio con features...
Filas: 10706


## Definición de variables de entrada (X) y salida (y)

A partir del dataset limpio, seleccionamos **únicamente las columnas numéricas** como variables de entrada (*features*), excluyendo explícitamente la columna objetivo `label`.

Pasos en esta celda:

1. Seleccionar todas las columnas numéricas.
2. Eliminar `label` de la lista de *features*.
3. Definir:
   - `X`: matriz de características numéricas.
   - `y`: variable objetivo binaria (0 = correo legítimo, 1 = fraude/phishing).
4. Dividir el dataset en:
   - **Train** (80%) para entrenar y hacer validación cruzada.
   - **Test** (20%) para evaluación final.
5. Usar `stratify=y` para mantener el mismo balance de clases en Train y Test.

In [2]:
# ===========================
# 1. Definir X, y
# ===========================
# Tomamos solo columnas numéricas y quitamos 'label'
num_cols = data.select_dtypes(include=[np.number]).columns.tolist()
num_cols.remove("label")

X = data[num_cols]
y = data["label"]

print("\nShapes:")
print("X:", X.shape)
print("y:", y.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print("\nTrain/Test:")
print("X_train:", X_train.shape, "X_test:", X_test.shape)




Shapes:
X: (10706, 13)
y: (10706,)

Train/Test:
X_train: (8564, 13) X_test: (2142, 13)


## Función auxiliar para reporte de métricas

Para evitar repetir código al evaluar cada modelo, definimos una función `print_metrics` que:

- Recibe:
  - Nombre del modelo (`nombre`)
  - Etiquetas reales (`y_true`)
  - Etiquetas predichas (`y_pred`)
  - Probabilidades predichas (`y_proba`)
- Imprime:
  - Accuracy
  - Precision
  - Recall
  - F1
  - AUC
  - **Classification report** completo (por clase)

Esto estandariza la comparación entre modelos.

In [4]:
# Función auxiliar para imprimir métricas
def print_metrics(nombre, y_true, y_pred, y_proba):
    print(f"\n=== {nombre} ===")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall   :", recall_score(y_true, y_pred))
    print("F1       :", f1_score(y_true, y_pred))
    print("AUC      :", roc_auc_score(y_true, y_proba))

    print("\nClassification report:")
    print(classification_report(y_true, y_pred))

## Modelo 1: Regresión Logística (tuneada con GridSearchCV)

En este bloque entrenamos una **Regresión Logística** y ajustamos sus hiperparámetros usando `GridSearchCV` con validación cruzada de 5 folds.

- Hiperparámetros explorados:
  - `C`: [0.01, 0.1, 1, 10] (fuerza de regularización)
  - `penalty`: "l2"
  - `solver`: ["lbfgs", "liblinear"]
- `scoring = "roc_auc"` para priorizar el área bajo la curva ROC.
- Al final:
  - Imprimimos los **mejores parámetros**.
  - Reportamos el **mejor AUC promedio en CV**.
  - Evaluamos el modelo óptimo sobre el conjunto de prueba (`X_test`, `y_test`) usando la función `print_metrics`.

In [5]:
# ===========================
# 2. Modelo 1: Logistic Regression (tuneada)
# ===========================
log_clf = LogisticRegression(max_iter=2000)

param_grid_log = {
    "C": [0.01, 0.1, 1, 10],
    "penalty": ["l2"],
    "solver": ["lbfgs", "liblinear"]
}

grid_log = GridSearchCV(
    log_clf,
    param_grid_log,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1
)

print("\n>>> Entrenando Logistic Regression (GridSearchCV)...")
grid_log.fit(X_train, y_train)
print("Mejores parámetros LogReg:", grid_log.best_params_)
print("Mejor AUC CV LogReg:", grid_log.best_score_)

best_log = grid_log.best_estimator_
y_pred_log = best_log.predict(X_test)
y_proba_log = best_log.predict_proba(X_test)[:, 1]

print_metrics("LOGISTIC REGRESSION TUNED", y_test, y_pred_log, y_proba_log)


>>> Entrenando Logistic Regression (GridSearchCV)...


/var/data/python/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 2000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=2000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/var/data/python/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 2000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=2000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Plea

Mejores parámetros LogReg: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
Mejor AUC CV LogReg: 0.8626755738396865

=== LOGISTIC REGRESSION TUNED ===
Accuracy : 0.7642390289449112
Precision: 0.8240887480190174
Recall   : 0.7860922146636432
F1       : 0.804642166344294
AUC      : 0.8550460205789003

Classification report:
              precision    recall  f1-score   support

           0       0.68      0.73      0.70       819
           1       0.82      0.79      0.80      1323

    accuracy                           0.76      2142
   macro avg       0.75      0.76      0.75      2142
weighted avg       0.77      0.76      0.77      2142



## Modelo 2: Random Forest (tuneado con GridSearchCV)

En este bloque ajustamos un **RandomForestClassifier** usando GridSearchCV:

- Hiperparámetros explorados:
  - `n_estimators`: [100, 300]
  - `max_depth`: [None, 10, 20]
  - `min_samples_split`: [2, 5]
  - `min_samples_leaf`: [1, 2]
- Mantenemos `random_state` fijo para reproducibilidad.
- Utilizamos nuevamente `scoring = "roc_auc"` y `cv = 5`.

Al final:

- Reportamos los parámetros óptimos.
- Evaluamos el modelo Random Forest resultante en el conjunto de prueba.
- Este modelo es un candidato fuerte para ser el **modelo principal** del proyecto.

In [6]:
# ===========================
# 3. Modelo 2: RandomForest (tuneado)
# ===========================
rf_clf = RandomForestClassifier(random_state=RANDOM_STATE)

param_grid_rf = {
    "n_estimators": [100, 300],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2]
}

grid_rf = GridSearchCV(
    rf_clf,
    param_grid_rf,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1
)

print("\n>>> Entrenando RandomForest (GridSearchCV)...")
grid_rf.fit(X_train, y_train)
print("Mejores parámetros RF:", grid_rf.best_params_)
print("Mejor AUC CV RF:", grid_rf.best_score_)

best_rf = grid_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)
y_proba_rf = best_rf.predict_proba(X_test)[:, 1]

print_metrics("RANDOM FOREST TUNED", y_test, y_pred_rf, y_proba_rf)


>>> Entrenando RandomForest (GridSearchCV)...
Mejores parámetros RF: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Mejor AUC CV RF: 0.9760134667750974

=== RANDOM FOREST TUNED ===
Accuracy : 0.9164332399626517
Precision: 0.9359756097560976
Recall   : 0.9281934996220711
F1       : 0.932068311195446
AUC      : 0.9727614285437415

Classification report:
              precision    recall  f1-score   support

           0       0.89      0.90      0.89       819
           1       0.94      0.93      0.93      1323

    accuracy                           0.92      2142
   macro avg       0.91      0.91      0.91      2142
weighted avg       0.92      0.92      0.92      2142



## Modelo 3: Gradient Boosting (tercer método de clasificación)

Como tercer modelo probamos **GradientBoostingClassifier**, un método de *boosting* que combina múltiples árboles débiles de manera secuencial.

- Hiperparámetros explorados:
  - `n_estimators`: [100, 200]
  - `learning_rate`: [0.05, 0.1]
  - `max_depth`: [3, 5]
- De nuevo utilizamos:
  - `cv = 5`
  - `scoring = "roc_auc"`

El objetivo es comparar su desempeño con Random Forest y Regresión Logística, y evaluar si el *boosting* aporta alguna mejora significativa en AUC y métricas de clasificación.

In [7]:
# ===========================
# 4. Modelo 3: Gradient Boosting (tercer método)
# ===========================
gb_clf = GradientBoostingClassifier(random_state=RANDOM_STATE)

param_grid_gb = {
    "n_estimators": [100, 200],
    "learning_rate": [0.05, 0.1],
    "max_depth": [3, 5]
}

grid_gb = GridSearchCV(
    gb_clf,
    param_grid_gb,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1
)

print("\n>>> Entrenando GradientBoosting (GridSearchCV)...")
grid_gb.fit(X_train, y_train)
print("Mejores parámetros GB:", grid_gb.best_params_)
print("Mejor AUC CV GB:", grid_gb.best_score_)

best_gb = grid_gb.best_estimator_
y_pred_gb = best_gb.predict(X_test)
y_proba_gb = best_gb.predict_proba(X_test)[:, 1]

print_metrics("GRADIENT BOOSTING TUNED", y_test, y_pred_gb, y_proba_gb)


>>> Entrenando GradientBoosting (GridSearchCV)...
Mejores parámetros GB: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
Mejor AUC CV GB: 0.973294383851621

=== GRADIENT BOOSTING TUNED ===
Accuracy : 0.919234360410831
Precision: 0.9369300911854104
Recall   : 0.9319727891156463
F1       : 0.9344448654793482
AUC      : 0.9716751712216565

Classification report:
              precision    recall  f1-score   support

           0       0.89      0.90      0.89       819
           1       0.94      0.93      0.93      1323

    accuracy                           0.92      2142
   macro avg       0.91      0.92      0.91      2142
weighted avg       0.92      0.92      0.92      2142



## Calibración de probabilidades: Platt vs Isotónica sobre Random Forest

Finalmente, realizamos un **análisis de calibración** sobre el modelo seleccionado (Random Forest):

- Usamos `CalibratedClassifierCV` con dos métodos:
  - **Platt (sigmoid)**: ajuste logístico sobre las probabilidades.
  - **Isotónica**: calibración no paramétrica más flexible.
- Entrenamos ambos calibradores sobre el conjunto de entrenamiento (`X_train`, `y_train`).
- Evaluamos las probabilidades calibradas en el conjunto de prueba mediante:
  - **Brier score** (mientras más bajo, mejor calibración).
  - **AUC** (para verificar que no se pierda demasiado desempeño).

Este análisis permite justificar cuál variante del modelo está **mejor calibrada** y es más adecuada para escenarios donde la probabilidad predicha es crítica (por ejemplo, priorizar correos a revisar manualmente).

In [8]:
# ===========================
# 5. Calibración (Platt vs Isotónica) sobre el mejor modelo
#    Aquí tomamos el RANDOM FOREST como candidato.
# ===========================
print("\n>>> Calibrando RandomForest (Platt / Isotonic)...")

cal_platt = CalibratedClassifierCV(best_rf, method="sigmoid", cv=5)
cal_platt.fit(X_train, y_train)

cal_iso = CalibratedClassifierCV(best_rf, method="isotonic", cv=5)
cal_iso.fit(X_train, y_train)

y_proba_platt = cal_platt.predict_proba(X_test)[:, 1]
y_proba_iso   = cal_iso.predict_proba(X_test)[:, 1]

brier_platt = brier_score_loss(y_test, y_proba_platt)
brier_iso   = brier_score_loss(y_test, y_proba_iso)

print("\nBrier score RF sin calibrar :", brier_score_loss(y_test, y_proba_rf))
print("Brier score RF Platt (sigmoid):", brier_platt)
print("Brier score RF Isotonic       :", brier_iso)
print("AUC RF Platt :", roc_auc_score(y_test, y_proba_platt))
print("AUC RF Iso   :", roc_auc_score(y_test, y_proba_iso))

print("\n Modelado tuneado + calibración COMPLETO.")


>>> Calibrando RandomForest (Platt / Isotonic)...

Brier score RF sin calibrar : 0.06233317159808155
Brier score RF Platt (sigmoid): 0.0617109837806013
Brier score RF Isotonic       : 0.06082338850997491
AUC RF Platt : 0.9720028019347746
AUC RF Iso   : 0.9720646364637295

 Modelado tuneado + calibración COMPLETO.
